# Tutorial 6 (World Advanced)

This is a tutorial for E-Cell4.

In [1]:
from ecell4.core import *
from ecell4.gillespie import GillespieWorld
from ecell4.ode import ODEWorld
from ecell4.spatiocyte import SpatiocyteWorld
from ecell4.bd import BDWorld

edge_lengths = Real3(1, 2, 3)

In [2]:
m = NetworkModel()
m.add_species_attribute(Species("A", "0.0025", "1"))

`ParticleSpace` interfaces:

In [3]:
w = BDWorld(edge_lengths)
# w = SpatiocyteWorld(edge_lengths)
w.bind_to(m)

Create a new particle. `new_particle` returns a pair of `ParticleID` and `Particle` with if the creation is succeeded or not.

In [4]:
((pid, p), is_succeeded) = w.new_particle(Species("A"), Real3(0.5, 0.5, 0.5))

In [5]:
print(pid, p, is_succeeded)
print(w.num_molecules(Species("A")))

(<ecell4.core.ParticleID object at 0x7fcb749399d8>, <ecell4.core.Particle object at 0x7fcb74939a20>, True)
1


`ParticleID` is a pair of `Integer`s named `serial` and `lot`. `Particle` consists of `species`, `position`, `radius`, `D`.

In [6]:
print(pid.serial(), pid.lot())
print(p.species().serial(), tuple(p.position()), p.radius(), p.D())

(1L, 0)
(u'A', (0.5, 0.5, 0.5), 0.0025, 1.0)


`ParticleSpace` has interfaces to check if the `ParticleID` exists (`has_particle`), to list all particles belonging to the `Species` (`list_particles`), and to get `Particle` assigned to the given `ParticleID` (`get_particle`).

In [7]:
print(w.has_particle(pid))
print(w.list_particles(Species("A")))
print(w.get_particle(pid))

True
[(<ecell4.core.ParticleID object at 0x7fcb749399f0>, <ecell4.core.Particle object at 0x7fcb74939a50>)]
(<ecell4.core.ParticleID object at 0x7fcb749399f0>, <ecell4.core.Particle object at 0x7fcb74939990>)


To move a `Particle` to the new position, use `update_particle`. `update_particle` tries to replace the `Particle` with given one.

In [8]:
w.update_particle(pid, Particle(p.species(), Real3(0.5, 1.5, 2.5), p.radius(), p.D()))
_, newp = w.get_particle(pid)
print(tuple(newp.position()), w.num_molecules(Species("A")))

((0.5, 1.5, 2.5), 1)


You can remove a `Particle` by `remove_particle`.

In [9]:
w.remove_particle(pid)
print(w.has_particle(pid))

False


In [10]:
del w

`LatticeSpace` interfaces:

In [11]:
w = SpatiocyteWorld(edge_lengths)
w.bind_to(m)

`LatticeSpace` has interfaces to give essential information about it.

In [12]:
print(w.voxel_radius(), w.row_size(), w.col_size(), w.layer_size())

(0.01, 152, 62, 116)


Positions in `LatticeSpace` is represented as a single `Integer` named `coordinate`, which is corresponding to `Real3` (`position`) in `ParticleSpace`. `coordinate2position` and `position2coordinate` give the way to convert between them.

In [13]:
coord = w.position2coordinate(Real3(0.5, 0.5, 0.5))
pos = w.coordinate2position(coord)
new_coord = w.position2coordinate(pos)

print(coord, tuple(pos))
print(new_coord, tuple(w.coordinate2position(new_coord)))
print(w.position2coordinate(Real3(0.5, 1.5, 2.5)))

(278033, (0.5062278801751902, 0.5080682368868706, 0.5))
(278033, (0.5062278801751902, 0.5080682368868706, 0.5))
815301


Interfaces similar to `ParticleSpace` are available in `LatticeSpace`.

In [14]:
((pid, v), is_succeeded) = w.new_voxel(Species("A"), coord)

In [15]:
print(pid, v, is_succeeded)
print(w.num_molecules(Species("A")))

(<ecell4.core.ParticleID object at 0x7fcb74939a38>, <ecell4.core.Voxel object at 0x7fcb74939a80>, True)
1


In [16]:
print(v.species().serial(), v.coordinate(), v.radius(), v.D())

(u'A', 278033, 0.0025, 1.0)


In [17]:
print(w.has_particle(pid))
print(w.list_voxels(Species("A")))
print(w.get_voxel(pid))

True
[(<ecell4.core.ParticleID object at 0x7fcb749399d8>, <ecell4.core.Voxel object at 0x7fcb74939a98>)]
(<ecell4.core.ParticleID object at 0x7fcb749399d8>, <ecell4.core.Voxel object at 0x7fcb74939a50>)


In [18]:
w.update_voxel(pid, Voxel(v.species(), 791525, v.radius(), v.D()))
_, newv = w.get_voxel(pid)
print(newv.coordinate(), w.num_molecules(Species("A")))

(791525, 1)


In [19]:
w.remove_voxel(pid)
print(w.has_voxel(pid))

False


`RandomNumberGenerator` interfaces:

In [20]:
rng1, rng2 = GSLRandomNumberGenerator(), GSLRandomNumberGenerator()
rng1.seed(0)
rng2.seed(0)
print(rng1.uniform(0, 1), rng2.uniform(0, 1))
print(rng1.uniform_int(0, 100), rng2.uniform_int(0, 100))
print(rng1.gaussian(0.0, 1.0), rng2.gaussian(0.0, 1.0))

(0.999741748906672, 0.999741748906672)
(16, 16)
(1.0, 1.0)


`GillespieWorld`, `SpatiocyteWorld` and `BDWorld` can be constructed with a `GSLRandomNumberGenerator`. `rng()` returns a shared object of `RandomNumberGenerator`. In below, `w.rng()` and `rng1` point the same `RandomNumberGenerator`.

In [21]:
w = GillespieWorld(Real3(1, 1, 1), rng1)
# w = BDWorld(Real3(1, 1, 1), rng1)
# w = SpatiocyteWorld(Real3(1, 1, 1), 0.05, rng1) # The second argument is voxel_radius.

In [22]:
print(w.rng().uniform(0, 1), rng2.uniform(0, 1), rng1.uniform(0, 1))

(0.23165654274635017, 0.23165654274635017, 0.48497361433692276)


In [23]:
del rng1, rng2
print(w.rng().uniform(0, 1))
del w

0.957476956537
